In [38]:
import numpy as np

# <center> TP3 - Chiffrement de Hill </center>
<center> 2025/2026 - L. Naert, S. Bouchelaghem, T. Godin </center>

_Certains exemples et textes de ce TP sont tirés de Exercices et Problèmes de cryptographie de Damien Vergnaud, 3ème édition_

__Usage de l'IA générative : interdit__

Dans ce TP, nous étudierons le chiffrement de Hill (chiffrement et déchiffrement). L'attaque de ce chiffrement ne sera pas étudiée.

Par convention, nous appellerons :
- $k$ : la clef
- $E_k$ : la fonction de chiffrement
- $D_k$ : la fonction de déchiffrement
- $m$ : le message en clair
- $m_i$ : la lettre de rang $i$ sur message en clair
- $c$ : le message chiffré
- $c_i$ : la lettre de rang $i$ sur message chiffré

_Note_ : à tout moment, vous pouvez utiliser des fonctions définies dans les TPs précédents.

In [39]:
# Quelques fonctions utiles (ou pas)
def lettreToEntier(lettre, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    return alphabet.find(lettre)
def entierToLettre(a, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    return alphabet[int(a)]

print("Lettre -> entier : w ->", lettreToEntier('w'))
print("Entier -> Lettre : 22 ->", entierToLettre(22))

Lettre -> entier : w -> 22
Entier -> Lettre : 22 -> w


## Chiffrement de Hill

Le chiffrement de Hill est un type de chiffrement par substitution polyalphabétique où les lettres du clair sont chiffrées et déchiffrées par paquets et non les unes à la suite des autres. Ce chiffrement est donc mieux protégé contre les attaques par analyse de fréquence.

Pour chiffrer, on commence par choisir une matrice carrée inversible (nous verrons plus tard comment vérifier qu'une matrice est inversible dans $\mathbb{Z}/26\mathbb{Z}$) de taille $p \times p$. Cette matrice constitue la clef de chiffrement. Ensuite, le message en clair est divisé en blocs/vecteurs de longueur $p$. Le dernier bloc est éventuellement complété avec une lettre choisie arbitrairement si sa longueur est différente de $p$. Chaque vecteur est chiffré en le multipliant avec la matrice carré. Evidemment, pour effectuer la multiplication matricielle, les lettres des messages sont converties en nombre entre 0 et 25. 

La fonction de chiffrement correspondante pour un bloc de $p$ lettres est : 

\begin{align*}
  E \colon (\mathbb{Z}/26\mathbb{Z})^p &\to (\mathbb{Z}/26\mathbb{Z})^p\\
    \begin{pmatrix}
m_i\\
m_{i+1}\\
... \\
m_{i+p-1} \\
\end{pmatrix} & \mapsto \begin{pmatrix}
c_i\\
c_{i+1}\\
... \\
c_{i+p-1} \\
\end{pmatrix} = K \times     \begin{pmatrix}
m_i\\
m_{i+1}\\
... \\
m_{i+p-1} \\
\end{pmatrix}
\end{align*}

où $K$ est une matrice carrée inversible de taille $p \times p$.

Par exemple, Chiffrons le message "TEXTE" avec une matrice $K \in \mathcal{M_{2,2}}$ : 
$\begin{pmatrix}
3 & 5\\
6 & 17\\
\end{pmatrix}$.

Nous commençons par faire des blocs de 2 lettres : "TE","XT","EW" (ici, on rajoute un W en fin de message pour avoir un dernier bloc de deux lettres car le message initial n'a pas un nombre pair de lettres). 

"TE" correspond au couple de valeurs (19,4). 


$
\begin{pmatrix}
3 & 5\\
6 & 17\\
\end{pmatrix} \times
\begin{pmatrix}
19\\
4\\
\end{pmatrix} = 
\begin{pmatrix}
77\\
182\\
\end{pmatrix}
$


Il faut ensuite convertir le résultat $\begin{pmatrix}
77\\
182\\
\end{pmatrix}$ en nombre de $\mathbb{Z}/26\mathbb{Z}$ :$\begin{pmatrix}
25\\
0\\
\end{pmatrix}$ puis en lettre : $\begin{pmatrix}
Z\\
A\\
\end{pmatrix}.$
"TE" sera donc chiffré "ZA". 
En recommançant le processus pour les autres couples de lettres, le clair "TEXTE" devient le chiffré "ZAITSI".

### a) Chiffrement

> __Question 1 (chiffrement)__ : Définir une fonction `chiffrementHill(msgClair, clef, lettreRemplacement, alphabet)` qui étant donné un message en clair `msgClair`, une clef de chiffrement `clef` sous forme de matrice carré $p \times p$ (np.array), une lettre de "padding" (par défaut "w") s'il n'est pas possible de diviser le message clair en blocs de taille p et un alphabet `alphabet` (par défaut, l'alphabet français), renvoie le message chiffré correspondant.

_Remarque : voir le try/assert pour avoir le format des entrées._

In [40]:
def chiffrementHill(msgClair, clef, lettrePadding='w', alphabet="abcdefghijklmnopqrstuvwxyz"):
    n = clef.shape[0]

    while len(msgClair) % n != 0:
        msgClair += lettrePadding

    msgChiffre = ""

    for i in range(0, len(msgClair), n):
        bloc = np.array([[lettreToEntier(msgClair[i + j], alphabet)] for j in range(n)])

        blocChiffre = np.dot(clef, bloc) % len(alphabet)

        for val in blocChiffre:
            msgChiffre += entierToLettre(int(val), alphabet)

    return msgChiffre


print(chiffrementHill("texte",np.array([[3,5],[6,17]])))
try:
    assert chiffrementHill("texte",np.array([[3,5],[6,17]])) == "zaitsi"
    assert chiffrementHill("chiffredehill",np.array([[1,3,1],[1,1,0],[2,9,3]])) == "fjnlkcrhvqppvha"
    print("chiffrementHill : OK")
except:
    print("chiffrementHill : ERREUR")

zaitsi
chiffrementHill : OK


C:\Users\marin\AppData\Local\Temp\ipykernel_7616\1062132384.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  msgChiffre += entierToLettre(int(val), alphabet)


### b) Déchiffrement


Partant du message chiffré $c$ et connaissant la matrice de chiffrement de taille $p \times p $, il est possible de déchiffrer le message pour obtenir le clair $m$.

Il s'agit de diviser le message chiffré en vecteurs de $p$ lettres et de multiplier ces vecteurs par l'__inverse__ de la matrice de chiffrement.

La fonction de déchiffrement d'un bloc de $p$ lettres est ainsi : 

\begin{align*}
  D \colon (\mathbb{Z}/26\mathbb{Z})^p &\to (\mathbb{Z}/26\mathbb{Z})^p\\
    \begin{pmatrix}
c_i\\
c_{i+1}\\
... \\
c_{i+p-1} \\
\end{pmatrix} & \mapsto \begin{pmatrix}
m_i\\
m_{i+1}\\
... \\
m_{i+p-1} \\
\end{pmatrix} = K^{-1} \times     \begin{pmatrix}
c_i\\
c_{i+1}\\
... \\
c_{i+p-1} \\
\end{pmatrix}
\end{align*}

où $ K^{-1}$ est l'inverse de la matrice de chiffrement K dans $\mathbb{Z}/26\mathbb{Z}$.


Pour déchiffrer le message, nous avons donc besoin d'inverser la matrice K dans $\mathbb{Z}/26\mathbb{Z}$. 


__La matrice K est inversible dans $\mathbb{Z}/n\mathbb{Z}$ si son déterminant est inversible dans $\mathbb{Z}/n\mathbb{Z}$.__





> __Question 2 :__
- Faire une fonction `estInversibleMat(k, n)` qui renvoie `True` si la matrice carrée $k$ (de type np.array) est inversible sur $\mathbb{Z}/n\mathbb{Z}$ et `False`sinon.

_Note : vous pouvez utiliser `np.linalg.det(mat)` pour obtenir le déterminant de `mat`. Attention python renvoie parfois un nombre décimal pour ce déterminant. Il faut donc arrondir le résultat (`np.round(x)`)_

In [41]:
import math
def estInversibleMat(k,n):
    return math.gcd(round(np.linalg.det(k)), n) == 1 

try:
    assert estInversibleMat(np.array([[3,5],[6,17]]),26) == True
    assert estInversibleMat(np.array([[1,3,1],[1,1,0],[2,9,3]]),26) == True
    assert estInversibleMat(np.array([[1,2],[3,4]]),26) == False
    print("estInversibleMat : OK")
except: 
    print("estInversibleMat : ERREUR")


estInversibleMat : OK


__Pour inverser K, il faut multiplier la transposée de la comatrice (= matrice des cofacteurs) de K par un inverse modulaire de son déterminant.__


> __Question 3 (comatrice) :__
> La fonction `reductionMatrice(mat,rowNum,colNum)` renvoie la matrice donnée en paramètre (sous forme de np.array) à laquelle on a retiré la ligne `rowNum` et la colonne `colNum`. En utilisant cette fonction, écrire la fonction `comatrice(mat)` qui renvoie la matrice des cofacteurs de `mat`. 

In [42]:
def reductionMatrice(mat,rowNum,colNum):
    n, m = mat.shape
    if(rowNum == 0 ) :
        B = mat[rowNum+1:]
    elif (rowNum == n-1) :
        B = mat[:rowNum]
    else :
        B = np.concatenate((mat[:rowNum],mat[rowNum+1:]))

    if(colNum == 0 ) :
        cofacteur = B[:,colNum+1:]
    elif (colNum == m-1) :
        cofacteur = B[:,:colNum]
    else :
        cofacteur = np.concatenate((B[:,:colNum],B[:,colNum+1:]))
    cofacteur = np.reshape(cofacteur, (n-1,m-1))
    cofacteur = np.transpose(cofacteur)
    return cofacteur

A = np.array([[1, 2, 3], [4, 5,6],[7,8,9]])
print(A)
print("Matrice A après retrait de la ligne 2 et colonne 1 (les indices commencent à 0) :\n", reductionMatrice(A, 2,1))
 


[[1 2 3]
 [4 5 6]
 [7 8 9]]
Matrice A après retrait de la ligne 2 et colonne 1 (les indices commencent à 0) :
 [[1 3]
 [4 6]]


In [43]:
def comatrice(mat):
    n, m = mat.shape
    comat = np.zeros((n,m), dtype=int)
    for i in range(n):
        for j in range(m):
            cofacteur = reductionMatrice(mat,i,j)
            signe = (-1)**(i+j)
            comat[i][j] = signe * round(np.linalg.det(cofacteur))
    return comat

try:
    A = np.array([[1, 2, 3], [4, 5,6],[7,8,9]])
    assert (comatrice(np.array([[3,5],[6,17]])) == np.array([[17, -6], [-5, 3]])).all()
    assert (comatrice(A) ==  np.array([[-3,6,-3], [6,-12,6], [-3,6,-3]])).all()
    print("comatrice : OK")
except:
    print("comatrice : ERREUR")

comatrice : OK


> __Question 4 (matrice inverse) :__
> A partir de la fonction précédente, faire une fonction `inverseMat(k, n)` qui teste si la matrice k (de type np.array) est inversible. Si oui, renvoie la matrice inverse de k dans $\mathbb{Z}/n\mathbb{Z}$ et sinon, renvoie -1. N'hésitez pas à réutiliser des fonctions du TP précédent.

In [44]:
# sources : cette [vidéo](https://www.youtube.com/watch?v=7o79t2KAKxE&list=PLE8WtfrsTAinMMyQkK_CzXhXU_LHRNXy_&index=3) 
# et [celle-ci](https://www.youtube.com/watch?v=BkK1_FspgYQ).
def coeffBezout(a, b) :
    l1 = np.array([a, 1, 0])
    l2 = np.array([b, 0, 1])
    
    while( l2[0] != 0):
        mult = l1[0]//l2[0]
        ltmp = l2
        l2 = l1 - mult*l2
        l1 = ltmp
    if(l1[0]<0):
        l1 = -l1
        
    return l1.tolist()

def inverse(a, n) : 
    bezout = coeffBezout(a, n)
    if bezout[0] != 1 : 
        return - 1
    else : 
        return bezout[1] % n

def inverseMat(k,n):
    det = round(np.linalg.det(k))
    detInv = inverse(det,n)
    if detInv == -1 :
        return -1
    comat = comatrice(k)
    adj = np.transpose(comat)
    invMat = (detInv * adj) % n
    return invMat


    

try:
    assert (inverseMat(np.array([[3,5],[6,17]]),26) == np.array([[7, 1],[22, 15]])).all()
    assert (inverseMat(np.array([[9,4],[5,7]]),26) == np.array([[ 5, 12],[15, 25]])).all()
    assert (inverseMat(np.array([[3,7],[5,8]]),26) == np.array([[ 4, 3],[17, 21]])).all()
    assert (inverseMat(np.array([[1,3,1],[1,1,0],[2,9,3]]),26) == np.array([[ 3,0,25],[23,1,1],[ 7,23,24]])).all()
    print("inverseMat : OK")
except:
    print("inverseMat : ERREUR")

inverseMat : OK


> __Question 5 (déchiffrement)__ : Définir une fonction `dechiffrementHill(msgChiffre, clef, alphabet)` qui étant donné un message chiffré `msgChiffre`, la matrice de chiffrement `clef` qui a servi à construire ce message chiffré et un alphabet `alphabet` (par défaut, l'alphabet français) renvoie le message en clair correspondant.
>
> __Contrainte : Pour tout chiffrement symétrique, il est possible d'appeler la fonction de chiffrement pour définir la fonction de déchiffrement. Faites-le ici.__

In [45]:
def dechiffrementHill(msgChiffre, clef, alphabet = "abcdefghijklmnopqrstuvwxyz"):
    n = clef.shape[0]
    clef_inv = inverseMat(clef, len(alphabet))

    msgClair = ""

    for i in range(0, len(msgChiffre), n):
        bloc = np.array([[lettreToEntier(msgChiffre[i + j], alphabet)] for j in range(n)])

        blocClair = np.dot(clef_inv, bloc) % len(alphabet)

        for val in blocClair:
            msgClair += entierToLettre(int(val), alphabet)

    return msgClair


try:
    assert dechiffrementHill("zaitsi",np.array([[3,5],[6,17]])) == "textew"
    assert dechiffrementHill("fjnlkcrhvqppvha",np.array([[1,3,1],[1,1,0],[2,9,3]])) == "chiffredehillww"
    print("dechiffrementHill : OK")
except:
    print("dechiffrementHill : ERREUR")

dechiffrementHill : OK


C:\Users\marin\AppData\Local\Temp\ipykernel_7616\2453573600.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  msgClair += entierToLettre(int(val), alphabet)


> __Activité__ :
> 1. Envoyez un message chiffré grâce à Hill à votre voisin(e). Connaissant la clef de chiffrement, celui-ci doit le déchiffrer.
> 2. Déchiffrez un message envoyé par votre voisin(e) en utilisant la clef de chiffrement qu'il vous aura communiqué (clef différente de la précédente).